# Задача 4. Общая задача.

Можно ли узнать возраст клиента на основе информации о его расходах по карте? Мы подготовили задачу на базе реальных банковских транзакций. Совершенствуя свои продукты, банк использует информацию о пользователях, в том числе и возраст. Это помогает сделать персонализированные продукты, которые удовлетворяют реальные потребности клиентов. Но всегда ли календарный возраст соответствует образу жизни (и покупок) человека?

Ваша задача — по информации о расходах клиента банка предсказать, в какую из возрастных групп он попадает. Даны обучающие train данные для построения признаков и обучения моделей, и тестовые test данные для проверки алгоритмов. Это специальным образом подготовленная и анонимизированная информация, на которой можно обучать модели, сохраняя полную безопасность реальных данных клиентов. Решением задачи являются предсказания алгоритмов на тестовых данных. Чем меньше ошибок совершит ваш алгоритм, тем выше вы поднимитесь в рейтинге!

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import os

# os.chdir(r"./drive/MyDrive/Data Science для школьников/Data science 2021 2022 - продвинутый курс/1 домашка/data")
url = os.getcwd()
files = os.listdir(url)
files

['transactions_test.csv',
 'test.csv',
 'train_target.csv',
 'small_group_description.csv',
 'transactions_train.csv']

In [14]:
import pandas as pd

# Обучающий transactions_train.csv, в котором для каждой транзакции известна дата, сумма, тип и id клиента

transactions_train = pd.read_csv('transactions_train.csv')
transactions_train

,client_id,trans_date,small_group,amount_rur
0,33172,6,4,71.463
1,33172,6,35,45.017
2,33172,8,11,13.887
3,33172,9,11,15.983
4,33172,10,11,21.341
...,...,...,...,...
26450572,43300,727,25,7.602
26450573,43300,727,15,3.709
26450574,43300,727,1,6.448
26450575,43300,727,11,24.669


In [15]:
import pandas as pd

train_target = pd.read_csv('train_target.csv')
train_target

# у нас два датасета с признаками и другой с ответами?
# что первое приходит в голову чтобы сделать?

,client_id,bins
0,24662,2
1,1046,0
2,34089,2
3,34848,1
4,47076,3
...,...,...
29995,14303,1
29996,22301,2
29997,25731,0
29998,16820,3


In [57]:
train_target['bins'].value_counts() # соотношение классов

2    7560
3    7509
1    7497
0    7434
Name: bins, dtype: int64

In [56]:
import pandas as pd

descr = pd.read_csv('small_group_description.csv')
descr

,small_group,small_group_code
0,Зоотовары,0
1,Сетевые супермаркеты и продуктовые магазины,1
2,Обслуживание существующего автомобиля (АЗС),2
3,Оплата телефона и связи,3
4,Аптеки,4
...,...,...
199,Услуги починки (металлические изделия),199
200,Телеграф,200
201,Мануальные терапевты,201
202,Скорая помощь,202


In [ ]:
import pandas as pd

transactions_test = pd.read_csv('transactions_test.csv')
transactions_test

In [ ]:
import pandas as pd

test = pd.read_csv('test.csv')
test

In [58]:
print(train_target.shape)
print(transactions_train.shape)

# transactions_train
# Петя - покупка 1 - аптека
# Петя - покупка 2 - продукты
# Петя - покупка 3 - аптека
# ...
# Петя - покупка_апетека (2) - покупка_продукты (1)

# Петя - 3 (от 50 до 60 лет) - train_target

(30000, 2)
(26450577, 4)


In [59]:
len(transactions_train['client_id'].unique())

30000

In [35]:
transactions_train.isna().sum()

client_id      0
trans_date     0
small_group    0
amount_rur     0
dtype: int64

In [61]:
# есть датафремя разной длинны, что делать дальше?

# в датафрейме transactions_train одному клиенту соответсвует много покупок
# можно посчитать среднее каждой покупки и таким образов для одного клиента - 
# будет одна строчка со средними значениями всех покупок

# создаем новые признаки из колонки: amount_rur - сумма транзакции
new_features = transactions_train.groupby('client_id')['amount_rur'].agg(['sum', 'mean','std', 'min', 'max']).reset_index()
new_features.head()

# Петя - покупка 1 - сумма 1
# Петя - покупка 2 - сумма 2
# Петя - покупка 3 - сумма 3

# Петя - сумму всю которую он потралил - мин сумма - мах сумма - среднее - общее кол-во покупок за все время

,client_id,sum,mean,std,min,max
0,4,28404.121,39.450168,73.511624,0.043,1341.802
1,6,15720.739,21.535259,26.200397,0.045,315.781
2,7,53630.036,69.379089,253.261383,0.043,4505.971
3,10,34419.365,48.752642,63.191701,0.045,654.893
4,11,26789.404,32.991877,107.395139,0.388,2105.058


In [63]:
new_features.shape

(30000, 6)

In [47]:
# можно создать еще один признак - количество покупок для каждого клиента

cnt_for_each_client = transactions_train['client_id'].value_counts()
# добавить в датафрейм новый столбец
cnt_for_each_client

29644    1150
36143    1150
46196    1150
11395    1150
3879     1150
         ... 
49281     700
42252     700
25911     700
20867     700
30002     700
Name: client_id, Length: 30000, dtype: int64

In [48]:
cnt_for_each_client = cnt_for_each_client.reset_index()
cnt_for_each_client = cnt_for_each_client.rename(columns={"index": "client_id", "client_id": "number_trans"})
cnt_for_each_client

,client_id,number_trans
0,29644,1150
1,36143,1150
2,46196,1150
3,11395,1150
4,3879,1150
...,...,...
29995,49281,700
29996,42252,700
29997,25911,700
29998,20867,700


In [38]:
# Посчитаем для каждого клиента количество транзакций по каждой категории

counter_df_train = transactions_train.groupby(['client_id','small_group'])['amount_rur'].count()

counter_df_train

client_id  small_group
4          1              447
           2                1
           3               44
           4               93
           8                1
                         ... 
49998      50               8
           55               7
           78              13
           112              1
           131              1
Name: amount_rur, Length: 1255191, dtype: int64

In [39]:
cat_counts_train = counter_df_train.reset_index().pivot(index='client_id', columns='small_group',values='amount_rur')

cat_counts_train = cat_counts_train.fillna(0)
cat_counts_train.columns = ['small_group_'+str(i) for i in cat_counts_train.columns]
cat_counts_train.head()

,small_group_0,small_group_1,small_group_2,small_group_3,small_group_4,small_group_5,small_group_6,small_group_7,small_group_8,small_group_9,small_group_10,small_group_11,small_group_12,small_group_13,small_group_14,small_group_15,small_group_16,small_group_17,small_group_18,small_group_19,small_group_20,small_group_21,small_group_22,small_group_23,small_group_24,small_group_25,small_group_26,small_group_27,small_group_28,small_group_29,small_group_30,small_group_31,small_group_32,small_group_33,small_group_34,small_group_35,small_group_36,small_group_37,small_group_38,small_group_39,...,small_group_162,small_group_163,small_group_164,small_group_165,small_group_166,small_group_167,small_group_168,small_group_169,small_group_170,small_group_171,small_group_172,small_group_173,small_group_174,small_group_175,small_group_176,small_group_177,small_group_178,small_group_179,small_group_180,small_group_181,small_group_182,small_group_183,small_group_184,small_group_185,small_group_186,small_group_187,small_group_188,small_group_189,small_group_190,small_group_191,small_group_192,small_group_193,small_group_195,small_group_196,small_group_197,small_group_198,small_group_199,small_group_200,small_group_202,small_group_203
client_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,0.0,447.0,1.0,44.0,93.0,0.0,0.0,0.0,1.0,13.0,1.0,19.0,0.0,0.0,0.0,37.0,0.0,0.0,5.0,0.0,2.0,0.0,0.0,3.0,6.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,9.0,0.0,12.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2.0,397.0,0.0,172.0,10.0,0.0,0.0,0.0,0.0,6.0,3.0,9.0,0.0,0.0,0.0,103.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2.0,79.0,5.0,27.0,19.0,1.0,0.0,2.0,1.0,39.0,0.0,187.0,136.0,5.0,0.0,15.0,3.0,0.0,64.0,3.0,0.0,3.0,2.0,7.0,15.0,55.0,0.0,0.0,6.0,0.0,0.0,1.0,1.0,2.0,8.0,5.0,16.0,5.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,12.0,309.0,1.0,71.0,65.0,0.0,0.0,0.0,3.0,19.0,0.0,58.0,0.0,0.0,0.0,40.0,0.0,0.0,6.0,2.0,0.0,0.0,0.0,3.0,26.0,11.0,0.0,0.0,6.0,0.0,1.0,2.0,0.0,0.0,8.0,0.0,27.0,0.0,0.0,12.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,2.0,423.0,0.0,59.0,23.0,3.0,0.0,0.0,0.0,10.0,4.0,107.0,0.0,1.0,0.0,7.0,1.0,0.0,22.0,0.0,1.0,1.0,18.0,3.0,14.0,17.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,10.0,14.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [64]:
print(cat_counts_train.shape)

print(new_features.shape)

print(cnt_for_each_client.shape)

(30000, 202)
(30000, 6)
(30000, 2)


In [50]:
print(cat_counts_train.columns)

print(new_features.columns)

print(cnt_for_each_client)

Index(['small_group_0', 'small_group_1', 'small_group_2', 'small_group_3',
       'small_group_4', 'small_group_5', 'small_group_6', 'small_group_7',
       'small_group_8', 'small_group_9',
       ...
       'small_group_192', 'small_group_193', 'small_group_195',
       'small_group_196', 'small_group_197', 'small_group_198',
       'small_group_199', 'small_group_200', 'small_group_202',
       'small_group_203'],
      dtype='object', length=202)
Index(['client_id', 'sum', 'mean', 'std', 'min', 'max'], dtype='object')
       client_id  number_trans
0          29644          1150
1          36143          1150
2          46196          1150
3          11395          1150
4           3879          1150
...          ...           ...
29995      49281           700
29996      42252           700
29997      25911           700
29998      20867           700
29999      30002           700

[30000 rows x 2 columns]


In [54]:
# теперь все три датафрейма и датафрейм с ответами по client_id можно объединить и будет новый датафрейм с признаками и ответами

# как?

all_new_fesatures = cat_counts_train.merge(new_features, on=["client_id"])
all_new_fesatures = all_new_fesatures.merge(cnt_for_each_client, on=["client_id"])
all_new_fesatures = all_new_fesatures.merge(train_target, on=["client_id"])
all_new_fesatures.shape

(30000, 210)

In [55]:
all_new_fesatures.head()

,client_id,small_group_0,small_group_1,small_group_2,small_group_3,small_group_4,small_group_5,small_group_6,small_group_7,small_group_8,small_group_9,small_group_10,small_group_11,small_group_12,small_group_13,small_group_14,small_group_15,small_group_16,small_group_17,small_group_18,small_group_19,small_group_20,small_group_21,small_group_22,small_group_23,small_group_24,small_group_25,small_group_26,small_group_27,small_group_28,small_group_29,small_group_30,small_group_31,small_group_32,small_group_33,small_group_34,small_group_35,small_group_36,small_group_37,small_group_38,...,small_group_169,small_group_170,small_group_171,small_group_172,small_group_173,small_group_174,small_group_175,small_group_176,small_group_177,small_group_178,small_group_179,small_group_180,small_group_181,small_group_182,small_group_183,small_group_184,small_group_185,small_group_186,small_group_187,small_group_188,small_group_189,small_group_190,small_group_191,small_group_192,small_group_193,small_group_195,small_group_196,small_group_197,small_group_198,small_group_199,small_group_200,small_group_202,small_group_203,sum,mean,std,min,max,number_trans,bins
0,4,0.0,447.0,1.0,44.0,93.0,0.0,0.0,0.0,1.0,13.0,1.0,19.0,0.0,0.0,0.0,37.0,0.0,0.0,5.0,0.0,2.0,0.0,0.0,3.0,6.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,9.0,0.0,12.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28404.121,39.450168,73.511624,0.043,1341.802,720,1
1,6,2.0,397.0,0.0,172.0,10.0,0.0,0.0,0.0,0.0,6.0,3.0,9.0,0.0,0.0,0.0,103.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15720.739,21.535259,26.200397,0.045,315.781,730,1
2,7,2.0,79.0,5.0,27.0,19.0,1.0,0.0,2.0,1.0,39.0,0.0,187.0,136.0,5.0,0.0,15.0,3.0,0.0,64.0,3.0,0.0,3.0,2.0,7.0,15.0,55.0,0.0,0.0,6.0,0.0,0.0,1.0,1.0,2.0,8.0,5.0,16.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53630.036,69.379089,253.261383,0.043,4505.971,773,0
3,10,12.0,309.0,1.0,71.0,65.0,0.0,0.0,0.0,3.0,19.0,0.0,58.0,0.0,0.0,0.0,40.0,0.0,0.0,6.0,2.0,0.0,0.0,0.0,3.0,26.0,11.0,0.0,0.0,6.0,0.0,1.0,2.0,0.0,0.0,8.0,0.0,27.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34419.365,48.752642,63.191701,0.045,654.893,706,3
4,11,2.0,423.0,0.0,59.0,23.0,3.0,0.0,0.0,0.0,10.0,4.0,107.0,0.0,1.0,0.0,7.0,1.0,0.0,22.0,0.0,1.0,1.0,18.0,3.0,14.0,17.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,10.0,14.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26789.404,32.991877,107.395139,0.388,2105.058,812,3


In [65]:
X = all_new_fesatures.drop(['bins', 'client_id'], axis=1)
y = all_new_fesatures['bins']

In [70]:
# Тренировочные и тестовые данные
# Построение и подбор модели?
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import xgboost as xgb
# from catboost import CatBoostClassifier
import time

models = [LogisticRegression(), DecisionTreeClassifier(), SVC()]
names = ["Logistic Regression", "Decision Tree", "SVM"]
for model, name in zip(models, names):
    print(name)
    start = time.time()
    for score in ["accuracy"]:
        print(score)
        print(" : ")
        print(cross_val_score(model, X, y, scoring=score, cv=6).mean())
    print(time.time() - start)

Logistic Regression
accuracy
 : 


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


0.4315666666666667
26.9471652507782
Decision Tree
accuracy
 : 
0.4658333333333333
10.557656526565552
SVM
accuracy
 : 
0.38503333333333334
2525.267828941345
